In [40]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

In [41]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: Ranger (Random Forest)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [42]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [43]:
train_model <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    seed = NULL
){
    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }

    model <- ranger::ranger(
            num.trees = 1000,
            case.weights = weight_fn(train_labels),
            classification = TRUE,
            x=x_train,
            y=train_labels
        )

    if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
            levels(test_labels) <- c(levels(test_labels), "Forb")
            }

    # create predictions (ranger)
        model_predictions <- predict(
            model, 
            x_test
        )$prediction %>% as.factor()

        # generate the confusion matrix

        confusion_matrix <- caret::confusionMatrix(
            model_predictions, 
            test_labels,
            mode = "everything"
        )

        # generate an id to uniquely identify the model
        #model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            #custom = file,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs3.log")

        # track what levels are associated with the UUID

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
        
        return(model)
}

In [44]:
base_paths <- c(
    "grd_raw_raw.csv",
    "grd_raw_corrected.csv",
    "img_raw_raw.csv",
    "img_indices_only.csv",# include veg indices
    "grd_indices_only.csv"
)
# 

In [45]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

In [46]:
test_p_weight <- calculate_posterior_weights()
test_p_weight 


$Abiotic
[1] 3757.3

$Forb
[1] 244.34

$Graminoid
[1] 3701.29

$Lichen
[1] 2236.96

$Moss
[1] 1491.57

$ShrubDecid
[1] 5196.25

$ShrubEvergreen
[1] 3425.14

$TreeBroadleaf
[1] 175.15

$TreeConifer
[1] 276.04

$Unknown
[1] 0

In [47]:
outlier_functions <- list(
    clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x){return(x)}# no transform
)

outlier_treatments <- c(
    "clip",
    "no_treatment"
)

preprocess_functions <- list(
    no_treatment = function(x){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    "balanced",
    "no_treatment",
    "posterior"
)

preprocessing_treatments <- c(
    "min_max",
    "robust",
    "standard",
    "no_treatment"
)

In [48]:
test_data <- read.csv("Data/gs/x_test/img_raw_raw.csv")
test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()

In [49]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x %>% as.factor()

    for(o_treatment in outlier_treatments){
        for(p_treatment in preprocessing_treatments){
            for(w_treatment in weight_treatments){

                print(p_treatment)
                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                } 

                rf_model <- train_model(
                    train_data, 
                    labels, 
                    test_data,
                    test_labels,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id
                )
            try({

                add_model_to_manifest(
                    model_id = model_id,
                    outlier = o_treatment,
                    preprocessing = p_treatment,
                    source = filepath,
                    weight = w_treatment,
                    logpath="./gs_manifest_3.csv"
                )

                results <- validate_model(
                    rf_model, 
                    save_path, 
                    outlier_processing = outlier_functions[[o_treatment]],
                    transform_type = preprocess_functions[[p_treatment]],
                )

                aggregated_results <- aggregate_results(save_path)

                plot_by_pft(
                    aggregated_results,
                    save_path = paste0(save_path, "aggregate.html"),
                    open = FALSE
                )
            })
            #
                write_validation_table(
                    aggregated_results,
                    save_path = paste0(save_path, "table.html"),
                    open = FALSE
                )
            }
        }
    }
}

[1] "min_max"


Warning message in confusionMatrix.default(model_predictions, test_labels, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


[1] "preprocessing raster at Data/Ground_Validation/Imagery/BisonGulchQuads.envi"
[1] "Converted to Data frame?"
[1] TRUE
[1] "Noisy columns removed"
[1] TRUE
[1] "Filtered"
[1] TRUE


Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



[1] "Imputing..."
       x              y          X402.593_5nm     X407.593_5nm   
 Min.   :-149   Min.   :63.81   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:-149   1st Qu.:63.81   1st Qu.:0.2291   1st Qu.:0.2512  
 Median :-149   Median :63.81   Median :0.4504   Median :0.4672  
 Mean   :-149   Mean   :63.81   Mean   :0.4246   Mean   :0.4416  
 3rd Qu.:-149   3rd Qu.:63.81   3rd Qu.:0.6008   3rd Qu.:0.6138  
 Max.   :-149   Max.   :63.81   Max.   :1.0000   Max.   :1.0000  
                                                                 
  X412.593_5nm     X417.593_5nm     X422.593_5nm     X427.593_5nm   
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.2049   1st Qu.:0.1972   1st Qu.:0.2072   1st Qu.:0.1893  
 Median :0.4047   Median :0.3887   Median :0.3928   Median :0.3705  
 Mean   :0.3840   Mean   :0.3723   Mean   :0.3775   Mean   :0.3568  
 3rd Qu.:0.5480   3rd Qu.:0.5343   3rd Qu.:0.5371   3rd Qu.:0.5151  
 Max.   :1.0000   Max.   :1.0000   Max. 

...

[1] "preprocessing raster at Data/Ground_Validation/Imagery/BisonGulchQuads.envi"
[1] "Converted to Data frame?"
[1] TRUE
[1] "Noisy columns removed"
[1] TRUE
[1] "Filtered"
[1] TRUE


Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



[1] "Imputing..."
  X402.593_5nm      X407.593_5nm      X412.593_5nm      X417.593_5nm     
 Min.   :-2.0453   Min.   :-2.1960   Min.   :-2.0159   Min.   :-1.99022  
 1st Qu.:-0.9419   1st Qu.:-0.9466   1st Qu.:-0.9402   1st Qu.:-0.93626  
 Median : 0.1244   Median : 0.1272   Median : 0.1084   Median : 0.08755  
 Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.00000  
 3rd Qu.: 0.8487   3rd Qu.: 0.8563   3rd Qu.: 0.8607   3rd Qu.: 0.86590  
 Max.   : 2.7716   Max.   : 2.7768   Max.   : 3.2335   Max.   : 3.35525  
                                                                         
  X422.593_5nm       X427.593_5nm       X432.593_5nm       X437.593_5nm     
 Min.   :-2.04524   Min.   :-1.94807   Min.   :-1.96447   Min.   :-1.93249  
 1st Qu.:-0.92248   1st Qu.:-0.91464   1st Qu.:-0.90646   1st Qu.:-0.89405  
 Median : 0.08311   Median : 0.07489   Median : 0.07191   Median : 0.06774  
 Mean   : 0.00000   Mean   : 0.00000   Mean   : 0.00000   Mean   : 0.00000  
 3rd 

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Error applying model - likely an empty file:



ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'as.data.frame': NA/NaN/Inf in foreign function call (arg 1)


In [ ]:
uuid::UUIDgenerate()

[1] "dab02a4b-83f9-464d-8dba-b61f99f07412"

In [ ]:
outlier_functions[[o_treatment]]

function (df, ignore_cols = NULL) 
{
    new_df <- df %>% as.data.frame()
    cache_path <- "./assets/clip_data.json"
    fences <- rjson::fromJSON(file = cache_path)
    used_cols <- colnames(df)
    if (!is.null(ignore_cols)) {
        used_cols <- setdiff(intersect(names(fences), colnames(df)), 
            ignore_cols)
    }
    for (column_name in used_cols) {
        upper_fence <- fences[column_name]$upper
        lower_fence <- fences[column_name]$lower
        high_outliers <- df[, column_name] > upper_fence
        low_outliers <- df[, column_name] < lower_fence
        new_df[high_outliers, column_name] = upper_fence
        new_df[low_outliers, column_name] = lower_fence
    }
    return(new_df)
}
<bytecode: 0x558eedc870b0>
<environment: 0x558eedbf21b0>

In [ ]:
preprocess_functions[[p_treatment]]

function (df, ignore_cols = NULL) 
{
    used_cols <- colnames(df)
    if (!is.null(ignore_cols)) {
        used_cols <- setdiff(used_cols, ignore_cols)
    }
    scaled_df <- df %>% as.data.frame()
    for (col in used_cols) {
        col_centered <- scaled_df[, col] - median(scaled_df[, 
            col], na.rm = TRUE)
        col_c_iqr <- stats::IQR(col_centered)
        if (!is.null(col_c_iqr) & !is.na(col_c_iqr)) {
            scaled_df[, col] <- (col_centered/col_c_iqr)
        }
        else {
            scaled_df[, col] <- col_centered
        }
    }
    return(scaled_df)
}
<bytecode: 0x558eeb9d47d8>